# Testing stuff for markov utility

In [20]:
import pandas as pd
import numpy as np

data_file_path = "./ModelData/"
transition_probs = "psi.csv"
transition_key = "psi_key.csv"
transition_data = pd.read_csv(data_file_path  + transition_probs)
transition_key_data = pd.read_csv(data_file_path  + transition_key)

In [3]:
transition_data

,Unnamed: 0,ab,ba
0,0,0.2777,0.1948


In [4]:
transition_key_data

,Unnamed: 0,column,start,end
0,0,ab,a,b
1,1,ba,b,a


In [82]:
n_periods = 12

transition_data_transposed = transition_data.transpose()
transition_data_transposed = transition_data.transpose()
transition_data_transposed.index.name = 'column'
transition_data_transposed.reset_index(
            inplace=True)

transition_data_use = pd.merge(
    transition_key_data,
    transition_data_transposed,
    on="column")
    
    # Set up reference for indices associated with names in transition matrix
reference = list(set(transition_data_use['start'].unique()) | 
                     set(transition_data_use['end'].unique()))
    
    # Initialize the transition matrix
transition = np.zeros((len(reference), len(reference)))
    
    # Add entries to transtion matrix:
for index, row in transition_data_use.iterrows():
    start = reference.index(row['start'])
    end = reference.index(row['end'])
    transition[start, end] = row[0]
    
for row in range(len(transition)):
    transition[row, row] = 1 - transition[row].sum()
    
n_transition = np.linalg.matrix_power(transition, n_periods)

result = pd.DataFrame(columns=['start','end','prob'])

for m in range(len(n_transition)):
    for n in range(len(n_transition)):
        if m == n:
            continue
        if n_transition[m, n] == 0:
            continue
        new_row = pd.Series(data={'start':reference[m], 'end':reference[n], 'prob':n_transition[m, n]})
        result = result.append(new_row, ignore_index=True)

        
new_routes = [row['start'] + '-' + row['end'] for index, row in result.iterrows()]
result = result.assign(routes=new_routes)

new_keys = result[['routes', 'start', 'end']]
new_probs = pd.DataFrame(data = [list(result['prob'])], columns = result['routes'])

In [83]:
new_keys

,routes,start,end
0,b-a,b,a
1,a-b,a,b


In [84]:
new_probs

routes,b-a,a-b
0,0.412084,0.587452


In [80]:
new_probs = pd.DataFrame(data = [list(result['prob'])], columns = result['routes'])

In [81]:
new_probs

routes,b-a,a-b
0,0.412084,0.587452


In [23]:
transition = np.zeros((2,2))

In [24]:
transition

array([[0., 0.],
       [0., 0.]])